In [1]:
import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import cartopy.io.shapereader as shpreader
import shapely.geometry
import pandas as pd
import datetime


In [42]:
def calc_time(hours, start_year = 1900):
    
    '''
    
    This function calculates the week number, year and hour of the day based on the hours that are stored as 'time' in the ERA dataset.
    
    Args:
        hours: the 'time' variable of the ERA5 dataset
        
    Returns:
        hour: Hour between 0 and 23, 0 presumable corresponds to midnight
        week: Week number based on the Gregorian calendar
        year: Year number based on the Gregorian calendar
        
    '''
    
    start_date = datetime.date(start_year, 1, 1)
    hour, week, year = [], [], []
    
    for hr in hours:
        
        hours_added = datetime.timedelta(hours = np.int(hr))
        date = start_date + hours_added
        hour.append(hr%24)
        week.append(date.isocalendar()[1])
        year.append(date.isocalendar()[0])
        
    return(hour, week, year)

In [3]:
def load_country_mask(COUNTRY, data_dir, filename):

    '''
    
    This function loads the country masks for the ERA5 data grid we have been using

    Args:
        COUNTRY (str): This must be a name of a country (or set of) e.g. 
            'United Kingdom','France','Czech Republic'
 
       data_dir (str): The parth for where the data is stored.
            e.g '/home/users/zd907959/'

        filename (str): The filename of a .netcdf file
            e.g. 'ERA5_1979_01.nc'

        nc_key (str): The string you need to load the .nc data 
            e.g. 't2m','rsds'

    Returns:
       MASK_MATRIX_RESHAPE (array): Dimensions [lat,lon] where there are 1's if 
           the data is within a country border and zeros if data is outside a 
           country border. 
           
    '''

    # first loop through the countries and extract the appropraite shapefile
    countries_shp = shpreader.natural_earth(resolution='10m',category='cultural',
                                            name='admin_0_countries')
    country_shapely = []
    for country in shpreader.Reader(countries_shp).records():
        if country.attributes['NAME_LONG'] == COUNTRY:
            print('Found country')
            country_shapely.append(country.geometry)
            
    # load in the data you wish to mask
    file_str = data_dir + filename
    dataset = Dataset(file_str,mode='r')
    lons = dataset.variables['longitude'][:]
    lats = dataset.variables['latitude'][:]
    dataset.close()

    LONS, LATS = np.meshgrid(lons,lats) # make grids of the lat and lon data
    x, y = LONS.flatten(), LATS.flatten() # flatten these to make it easier to 
    #loop over.
    points = np.vstack((x,y)).T
    MASK_MATRIX = np.zeros((len(x),1))
    # loop through all the lat/lon combinations to get the masked points
    for i in range(0,len(x)):
        my_point = shapely.geometry.Point(x[i],y[i]) 
        if country_shapely[0].contains(my_point) == True: 
            MASK_MATRIX[i,0] = 1.0 # creates 1s and 0s where the country is
    
    MASK_MATRIX_RESHAPE = np.reshape(MASK_MATRIX,(len(lats),len(lons)))
    return(MASK_MATRIX_RESHAPE)



In [4]:
def load_1d_data(data_dir, filename, nc_key):

    '''
    Args:
 
       data_dir (str): The parth for where the data is stored.
            e.g '/home/users/zd907959/'

        filename (str): The filename of a .netcdf file
            e.g. 'ERA5_1979_01.nc'

        nc_key (str): The string you need to load the .nc data 
            e.g. 't2m','rsds'

    Returns:
       data: dimensions [time, lat, lon]

        latitudes (array): array of latitudes

        longitudes (array): array of longitudes
    '''

    # load in the variable you want to extract

    file_str = data_dir + filename
    dataset = Dataset(file_str,mode='r')

    lons = dataset.variables['longitude'][:]
    lats = dataset.variables['latitude'][:]
    data = dataset.variables[nc_key][:] # data in shape [time,lat,lon]
    dataset.close()

    # get data in appropriate units for models
    if nc_key == 't2m':
        data = data-273.15 # convert to Kelvin from Celsius
    if nc_key == 'ssrd':
        data = data/3600. # convert Jh-1m-2 to Wm-2

    return(data)

In [20]:
def load_daily_national_data(data_dir, filename, country_mask, input_var, output_var):

    """
    Args:
        data_dir (str): The parth for where the data is stored.
            e.g '/home/users/zd907959/'

        filename (str): The filename of a .netcdf file
            e.g. 'ERA5_1979_01.nc'
            
        country_mask
        
        input_var: LIST of variables you want as inputs. Make sure they ar present in the dataset you are sending to the function
        
        output_var: single string of the output variable of interest. Options are t2m, wind10, wind100 (wind is calculated below from u and v)
    
    Returns:

        You have the choice to either compute just the input df, just the output df or both
        by commenting/ uncommenting accordingly
        Dataframe X (input) with time series of averaged over daily values
        Dataframe Y (output) with the hourly values stored in columns
        
    """
    
    file_str = data_dir + filename
    dataset = Dataset(file_str,mode='r') 
    columns = dataset.variables.keys()
    dataset.close()
    
    columns =  input_var# list of input variables you are interested in processing
    out_var = output_var # output variable that you are interested in ---> Set1 only
    
    df = pd.DataFrame(columns = columns)

    for col in columns: 
        
        data =  load_1d_data(data_dir, filename, col)
    
        # country mask the data
        country_masked_data = np.zeros(np.shape(data))
        for i in range(0,len(country_masked_data)):
            country_masked_data[i,:,:] = data[i,:,:]*country_mask

        # to make this a national timeseries average over the existing country points, replace 0 with nans
        country_masked_data[country_masked_data == 0.] = np.nan

        # spatially average
        country_timeseries = np.nanmean(np.nanmean(country_masked_data,axis=2),axis=1)
        
        df[col] = country_timeseries
               
    '''
    Uncomment the next 4 lines only if you want to convert the time column into 
        weeks number and year and include in your dataset
    ''' 
    
    #hours_total = load_1d_data(data_dir, filename, 'time')
    #hour, week, year = calc_time(hours_total)
    #df['week'] = week
    #df['year'] = year
    
    '''
    Uncomment the next 2 lines only if you want to convert u,v to wind speed magnitude
    
    ---> Set 1 only
    '''
    
    # df['wind10'] = np.sqrt(df['u10']**2 + df['v10']**2)
    # df['wind100'] = np.sqrt(df['u100']**2 + df['v100']**2)
    
    '''
    Input features are averaged to get daily values
    '''
    X = df.groupby(np.arange(len(df))//24).mean()

    '''
    Uncomment the next 4 lines if you also want to compute the output dataframe.
    24 output features need to be collected in Y
    We store the hourly values of the output variable into rows
    Each row is a day
    Each column is an hour
    
    ---> Set1 only
    '''

    #Y = pd.DataFrame()
    #for idx, df in enumerate(np.array_split(df,  24)):
    #    Y[idx] = df[out_var].values
    # return(X, Y)
    
    return X


In [18]:
# user input
data_dir = '/gws/pw/j05/cop26_hackathons/oxford/Data/ERA5_data_EU_domain/field_set_2/'
set_name = 'set_2'
inputs = ['tcc', 'tp']
output = 't2m'

In [12]:
# filename is only used for the country mask, so doesn't matter which year/ lat/ lon
filename = 'ERA5_1hr_field_%s_2000_01.nc'%set_name
country_mask = load_country_mask('United Kingdom', data_dir, filename)

Found country


In [13]:
# WARNING: takes a long time to compute.

years = np.arange(1980, 2001, 1)
X = pd.DataFrame()
Y = pd.DataFrame()

for year in years:
    print(year) # to see the progres of the loop
    for month in np.arange(1, 13, 1):
        if X.empty and Y.empty:
        #if X.empty:
            X, Y = load_daily_national_data(data_dir,'ERA5_1hr_field_%s_%d_%02d.nc'%(set_name, year, month), country_mask)
            #X = load_daily_national_data(data_dir,'ERA5_1hr_field_set_2_%d_%02d.nc'%(year, month), country_mask)
        else:
            df1, df2 = load_daily_national_data(data_dir,'ERA5_1hr_field_%s_%d_%02d.nc'%(set_name, year, month), country_mask)
            #df1 = load_daily_national_data(data_dir,'ERA5_1hr_field_set_2_%d_%02d.nc'%(year, month), country_mask)
            X = X.append(df1, ignore_index = True)
            Y = Y.append(df2, ignore_index = True)
            

1980


/opt/jaspy/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice


1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000


In [19]:
''' 
save the dataframes to a file 
'''

#X.to_csv('_set2.dat')
#Y.to_csv('_wind10.dat')

' \nsave the dataframes to a file \n'

In [83]:
# trying to add the nao data from https://www.ncdc.noaa.gov/teleconnections/nao/ but it is monthly.
# need to find a way to correctly repeat the rows to have daily values. 
# probably need to hardcode the number of days each month ? check the python datetime module

filename = 'nao.csv'
df_nao = pd.read_csv(filename, header = 1, index_col = None)
df_nao = df_nao.astype(str)

df_nao['year'] = df_nao['Date'].str[:4]
df_nao['month'] = df_nao['Date'].str[4:]
df_nao.drop(columns = ['Date'], inplace = True)
df_nao = df_nao.astype(float)


In [84]:
df_nao = df_nao.loc[df_nao['year'].isin(np.arange(1980, 2021))]
